In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import scrapbook as sb 
import pandas as pd
import numpy as np 
import seaborn as sns 
import numpy as np
from statistics import mean
import matplotlib.pyplot as plt

# Baseline 

In [3]:
books = sb.read_notebooks("./BaseLine_Model_Output")
baseLine_data = []
for nb in books.notebooks: 
    nbList=[nb.scraps['Stats Model MAE'].data,nb.scraps['Catboost MAE'].data]
    baseLine_data.append(nbList)
df = pd.DataFrame(baseLine_data, columns = ["Stats Model","Catboost"])
#baseLine_data = np.reshape(baseLine_data,(2,10))
display(df)
print(df.mean())

,Stats Model,Catboost
0,0.343250,0.077727
1,0.322866,0.064946
2,0.386471,0.090121
3,0.417323,0.155601
4,0.328342,0.143718
5,0.393020,0.088314
6,0.362717,0.103711
7,0.428908,0.104581
8,0.387171,0.071411
9,0.327260,0.093738


Stats Model    0.369733
Catboost       0.099387
dtype: float64


# GAN Model

In [4]:
book = sb.read_notebooks("./GAN_Output")
gan_data = []
gan_mse = []
for nb in book.notebooks:
    metrics = nb.scraps['GAN_1 Metrics'].data
    for i in range(1000):
        gan_mse.append(metrics[0][i])
    nbList = [nb.scraps['GAN Model MSE'].data,
              nb.scraps['GAN Model MAE'].data,
              nb.scraps['GAN Model Euclidean distance'].data,
              nb.scraps['GAN Model Manhattan Distance'].data]        
    gan_data.append(nbList)
    
df = pd.DataFrame(gan_data, columns = ['MSE','MAE','Euclidean Distance','Manhattan Distance'])
display(df.style)
print("MEAN:")
print(df.mean(axis = 0))
gan_data = np.array(gan_data)

,MSE,MAE,Euclidean Distance,Manhattan Distance
0,0.224886,0.286654,4.789396,29.238673
1,0.138727,0.257540,3.761624,26.269120
2,0.265569,0.303295,5.204587,30.936141
3,0.096678,0.232739,3.140175,23.739412
4,0.143463,0.273409,3.825324,27.887764
5,0.156120,0.249827,3.990489,25.482312
6,0.107298,0.226535,3.308129,23.106587
7,0.105239,0.241151,3.276325,24.597380
8,0.160859,0.263085,4.050606,26.834708
9,0.129555,0.259179,3.635162,26.436244


MEAN:
MSE                    0.152839
MAE                    0.259342
Euclidean Distance     3.898182
Manhattan Distance    26.452834
dtype: float64


# ABC_GAN Analysis

## ABC Pre-generator - Catboost 


In [5]:
book = sb.read_notebooks("./ABC_GAN_Catboost")
paramVal = [1,0.1,0.01]
abc_mae = [[] for i in range(3)]
abc_mae_skip = [[] for i in range(3)]
abc_mae_mean = [[] for i in range(3)]
abc_mae_skip_mean = [[] for i in range(3)]
abc_weights = [[] for i in range(3)]
prior_model = [[] for i in range(3)]
abc_pre_generator = [[] for i in range(3)]

for nb in book.notebooks:
    metrics1 = np.array(nb.scraps['ABC_GAN_1 Metrics'].data)
    metrics3 = np.array(nb.scraps['ABC_GAN_3 Metrics'].data)
    paramVar = float(nb.papermill_dataframe.iloc[0]['value'])
    
    #Divide data according to parameters 
    for i in range(3):
        if paramVar == paramVal[i]:
            for j in range(100):
                abc_mae[i].append(metrics1[1,j])
                abc_mae_skip[i].append(metrics3[1,j])
            abc_weights[i].append(nb.scraps['Skip Connection Weight'].data)
            prior_model[i].append(nb.scraps['Prior Model MSE'].data)
            abc_pre_generator[i].append(nb.scraps['ABC Pre-generator MSE'].data)
            abc_mae_skip_mean[i].append(mean(metrics3[1,:]))
            abc_mae_mean[i].append(mean(metrics1[1,:]))

In [6]:
for i in range(3):
    data = []
    for j in range(len(abc_weights[i])):
        data.append([paramVal[i],prior_model[i][j],
                     abc_pre_generator[i][j],abc_weights[i][j],abc_mae_mean[i][j],abc_mae_skip_mean[i][j]])
    
    df = pd.DataFrame(data, columns = ['Variance','Prior Model MAE',
                                       'ABC pre-generator MAE','Skip Node weight','ABC GAN MAE','ABC_GAN MAE (skip connection)'])
    display(df.round(5))
    print(df.mean(axis=0))
    print("-------------------------------------------------------------")

,Variance,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,1,0.08175,1.14402,0.67706,0.24144,0.22711
1,1,0.12537,1.17901,0.84510,0.26088,0.24889
2,1,0.15930,1.35492,0.96218,0.25470,0.24627
3,1,0.11766,0.83472,0.86231,0.27078,0.27839
4,1,0.20120,1.53223,0.19647,0.24995,0.25269
5,1,0.10357,0.93368,0.60734,0.22465,0.26580
6,1,0.10208,1.01596,0.98683,0.26044,0.24644
7,1,0.07172,0.94114,0.92982,0.21405,0.21928
8,1,0.24232,1.12395,0.25798,0.25240,0.28794
9,1,0.08963,1.18496,0.22481,0.22233,0.21726


Variance                         1.000000
Prior Model MAE                  0.129461
ABC pre-generator MAE            1.124458
Skip Node weight                 0.654989
ABC GAN MAE                      0.245161
ABC_GAN MAE (skip connection)    0.249005
dtype: float64
-------------------------------------------------------------


,Variance,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.1,0.11433,0.12711,0.07704,0.24489,0.26398
1,0.1,0.18189,0.19910,0.38519,0.27501,0.26845
2,0.1,0.06773,0.08859,0.38973,0.20589,0.20944
3,0.1,0.12855,0.14961,0.06259,0.21159,0.23800
4,0.1,0.24304,0.25729,0.46108,0.28814,0.28751
5,0.1,0.08148,0.09058,0.24821,0.21760,0.22616
6,0.1,0.06897,0.08378,0.53694,0.19305,0.19243
7,0.1,0.09379,0.10558,0.31672,0.20912,0.21530
8,0.1,0.07683,0.08479,0.31771,0.21533,0.20556
9,0.1,0.09655,0.10810,0.46972,0.20117,0.22464


Variance                         0.100000
Prior Model MAE                  0.115317
ABC pre-generator MAE            0.129453
Skip Node weight                 0.326494
ABC GAN MAE                      0.226179
ABC_GAN MAE (skip connection)    0.233148
dtype: float64
-------------------------------------------------------------


,Variance,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.01,0.07781,0.07788,0.00000,0.21825,0.19977
1,0.01,0.05547,0.05523,0.09143,0.22774,0.18915
2,0.01,0.07085,0.07124,0.00000,0.20751,0.19753
3,0.01,0.15915,0.15889,0.00000,0.24712,0.23677
4,0.01,0.07393,0.07361,0.22466,0.19955,0.20190
5,0.01,0.11786,0.11736,0.03973,0.23024,13.10767
6,0.01,0.12419,0.12356,0.04787,0.25047,24.39800
7,0.01,0.08595,0.08423,0.00000,0.21890,0.21072
8,0.01,0.11725,0.11693,0.24885,0.23724,0.23271
9,0.01,0.07515,0.07481,0.00000,0.20994,0.19772


Variance                         0.010000
Prior Model MAE                  0.095762
ABC pre-generator MAE            0.095374
Skip Node weight                 0.065254
ABC GAN MAE                      0.224697
ABC_GAN MAE (skip connection)    3.917194
dtype: float64
-------------------------------------------------------------


## ABC Pre-generator - Stats 


In [7]:
book = sb.read_notebooks("./ABC_GAN_Stats")
paramVal = [1,0.1,0.01]
abc_mae = [[] for i in range(3)]
abc_mae_skip = [[] for i in range(3)]
abc_mae_mean = [[] for i in range(3)]
abc_mae_skip_mean = [[] for i in range(3)]
abc_weights = [[] for i in range(3)]
prior_model = [[] for i in range(3)]
abc_pre_generator = [[] for i in range(3)]

for nb in book.notebooks:
    metrics1 = np.array(nb.scraps['ABC_GAN_1 Metrics'].data)
    metrics3 = np.array(nb.scraps['ABC_GAN_3 Metrics'].data)
    paramVar = float(nb.papermill_dataframe.iloc[0]['value'])
    
    #Divide data according to parameters 
    for i in range(3):
        if paramVar == paramVal[i]:
            for j in range(100):
                abc_mae[i].append(metrics1[1,j])
                abc_mae_skip[i].append(metrics3[1,j])
            abc_weights[i].append(nb.scraps['Skip Connection Weight'].data)
            prior_model[i].append(nb.scraps['Prior Model MSE'].data)
            abc_pre_generator[i].append(nb.scraps['ABC Pre-generator MSE'].data)
            abc_mae_skip_mean[i].append(mean(metrics3[1,:]))
            abc_mae_mean[i].append(mean(metrics1[1,:]))

In [8]:
for i in range(3):
    data = []
    for j in range(len(abc_weights[i])):
        data.append([paramVal[i],prior_model[i][j],
                     abc_pre_generator[i][j],abc_weights[i][j],abc_mae_mean[i][j],abc_mae_skip_mean[i][j]])
    
    df = pd.DataFrame(data, columns = ['Variance','Prior Model MAE',
                                       'ABC pre-generator MAE','Skip Node weight','ABC GAN MAE','ABC_GAN MAE (skip connection)'])
    display(df.round(5))
    print(df.mean(axis=0))
    print("-------------------------------------------------------------")

,Variance,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,1,0.24223,0.99940,0.99635,0.27037,0.23360
1,1,0.28663,1.19125,1.00000,0.27484,0.22987
2,1,0.25505,1.30307,0.99619,0.25619,0.31436
3,1,0.25217,1.06199,1.00000,0.23780,0.27920
4,1,0.24029,1.70515,0.99301,0.27039,0.25326
5,1,0.23649,1.10980,0.98162,0.25109,0.24382
6,1,0.33275,1.19835,0.98465,0.26515,0.28727
7,1,0.24465,1.03430,1.00000,0.23744,0.24412
8,1,0.32976,1.53234,0.99516,0.31144,0.26092
9,1,0.26877,1.50405,1.00000,0.23729,0.24053


Variance                         1.000000
Prior Model MAE                  0.268880
ABC pre-generator MAE            1.263971
Skip Node weight                 0.994697
ABC GAN MAE                      0.261201
ABC_GAN MAE (skip connection)    0.258694
dtype: float64
-------------------------------------------------------------


,Variance,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.1,0.22290,0.22514,0.56073,0.24708,0.31755
1,0.1,0.38615,0.38287,0.57990,0.31581,0.27134
2,0.1,0.18117,0.19920,0.65527,0.26338,0.22896
3,0.1,0.30758,0.31153,0.77783,0.27138,0.24257
4,0.1,0.34458,0.33966,0.60911,0.31061,0.28738
5,0.1,0.33496,0.35338,0.65151,0.28690,0.28600
6,0.1,0.27268,0.25931,0.54054,0.32492,0.24832
7,0.1,0.31116,0.30521,0.73658,0.24850,0.27129
8,0.1,0.23010,0.22624,0.56280,0.24652,0.21886
9,0.1,0.29679,0.33553,0.52794,0.27152,0.23941


Variance                         0.100000
Prior Model MAE                  0.288806
ABC pre-generator MAE            0.293807
Skip Node weight                 0.620222
ABC GAN MAE                      0.278663
ABC_GAN MAE (skip connection)    0.261168
dtype: float64
-------------------------------------------------------------


,Variance,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.01,0.24562,0.24494,0.15695,0.26876,0.27196
1,0.01,0.23342,0.23461,0.17637,0.32652,0.21256
2,0.01,0.24961,0.25078,0.17172,0.26085,0.24503
3,0.01,0.18738,0.18771,0.13672,0.24846,0.24836
4,0.01,0.33314,0.33410,0.14596,0.34341,0.40510
5,0.01,0.16787,0.16786,0.16095,0.25512,0.21705
6,0.01,0.27483,0.27385,0.18940,0.33990,0.27076
7,0.01,0.24932,0.25006,0.12253,0.29004,50.62289
8,0.01,0.36417,0.36314,0.15154,0.30586,124.05198
9,0.01,0.27643,0.27743,0.13916,0.23209,0.24345


Variance                          0.010000
Prior Model MAE                   0.258179
ABC pre-generator MAE             0.258448
Skip Node weight                  0.155129
ABC GAN MAE                       0.287101
ABC_GAN MAE (skip connection)    17.678914
dtype: float64
-------------------------------------------------------------
